<a href="https://colab.research.google.com/github/sushirito/Methylmercury/blob/main/TST_Boltzmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

# Lattice parameters
nx = 64  # number of nodes along x-axis
ny = 64  # number of nodes along y-axis
nsteps = 20000  # number of time steps
noutput = 100  # data output interval (data written to "/data")
nfluids = 1  # number of fluid components (choose 1 or 2)
tauA = 1  # relaxation time for fluid A
tauB = 1  # relaxation time for fluid B (will only be used if nfluids = 2)

# Shan-Chen parameters
gA = -4.7  # self-interaction strength of fluid A
gB = 0.  # self-interaction strength of fluid B (will only be used if nfluids = 2)
gAB = 6.  # interaction strength of fluids A and B (will only be used if nfluids = 2)
rho0 = 1.  # reference density for pseudopotential (usually set to 1)
rhol = 2.1  # initial liquid density
rhog = 0.15  # initial gas density
radius = 15.  # initial droplet radius

# Fixed parameters; DO NOT CHANGE
npop = 9  # number of populations
cx = np.array([0, 1, 0, -1, 0, 1, -1, -1, 1])  # x-components of lattice vectors
cy = np.array([0, 0, 1, 0, -1, 1, 1, -1, -1])  # y-components of lattice vectors
weight = np.array([4./9., 1./9., 1./9., 1./9., 1./9., 1./36., 1./36., 1./36., 1./36.])

# Arrays
g = np.zeros((nfluids, nfluids))  # interaction strengths
rho = np.zeros((nfluids, nx * ny))  # density
press = np.zeros(nx * ny)  # pressure
ux = np.zeros(nx * ny)  # x-component of fluid velocity
uy = np.zeros(nx * ny)  # y-component of fluid velocity
Fx = np.zeros((nfluids, nx * ny))  # x-component of Shan-Chen force
Fy = np.zeros((nfluids, nx * ny))  # y-component of Shan-Chen force
feq = np.zeros((nfluids, npop))  # equilibrium populations
forcing = np.zeros((nfluids, npop))  # force populations
tau = np.zeros(nfluids)  # relaxation times
f1 = np.zeros((nfluids, nx * ny, npop))  # populations (old)
f2 = np.zeros((nfluids, nx * ny, npop))  # populations (new)

def compute_density(pop):
    for s in range(nfluids):
        rho[s] = np.sum(pop[s], axis=1)

def psi(dens):
    return rho0 * (1. - np.exp(-dens / rho0))

def compute_sc_forces():
    for y in range(ny):
        for x in range(nx):
            k = y * nx + x
            for s in range(nfluids):
                Fx[s, k] = 0.
                Fy[s, k] = 0.
                for ss in range(nfluids):
                    fxtemp = 0.
                    fytemp = 0.
                    for i in range(1, npop):
                        x2 = (x + cx[i] + nx) % nx
                        y2 = (y + cy[i] + ny) % ny
                        psinb = psi(rho[ss, y2 * nx + x2])
                        fxtemp += weight[i] * cx[i] * psinb
                        fytemp += weight[i] * cy[i] * psinb
                    psiloc = psi(rho[s, k])
                    fxtemp *= (-g[s, ss] * psiloc)
                    fytemp *= (-g[s, ss] * psiloc)
                    Fx[s, k] += fxtemp
                    Fy[s, k] += fytemp

def compute_total_density(k):
    if nfluids == 1:
        return rho[0, k]
    elif nfluids == 2:
        return rho[0, k] + rho[1, k]

def compute_pressure():
    for y in range(ny):
        for x in range(nx):
            k = y * nx + x
            press[k] = 0.
            for s in range(nfluids):
                press[k] += rho[s, k] / 3.
                for ss in range(nfluids):
                    press[k] += (g[s, ss] * psi(rho[s, k]) * psi(rho[ss, k]) / 6.)

def compute_velocity(pop):
    for y in range(ny):
        for x in range(nx):
            k = y * nx + x
            ux[k] = 0.
            uy[k] = 0.
            for s in range(nfluids):
                ux[k] += (pop[s, k, 1] - pop[s, k, 3] + pop[s, k, 5] - pop[s, k, 6] - pop[s, k, 7] + pop[s, k, 8])
                uy[k] += (pop[s, k, 2] - pop[s, k, 4] + pop[s, k, 5] + pop[s, k, 6] - pop[s, k, 7] - pop[s, k, 8])
                ux[k] += (0.5 * Fx[s, k])
                uy[k] += (0.5 * Fy[s, k])
            dens = compute_total_density(k)
            ux[k] /= dens
            uy[k] /= dens

def equilibrium(s, k):
    dens = rho[s, k]
    vx = ux[k]
    vy = uy[k]
    usq = vx * vx + vy * vy
    feq[s, 0] = weight[0] * dens * (1. - 1.5 * usq)
    feq[s, 1] = weight[1] * dens * (1. + 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s, 2] = weight[2] * dens * (1. + 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s, 3] = weight[3] * dens * (1. - 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s, 4] = weight[4] * dens * (1. - 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s, 5] = weight[5] * dens * (1. + 3. * (vx + vy) + 4.5 * (vx + vy) * (vx + vy) - 1.5 * usq)
    feq[s, 6] = weight[6] * dens * (1. + 3. * (-vx + vy) + 4.5 * (-vx + vy) * (-vx + vy) - 1.5 * usq)
    feq[s, 7] = weight[7] * dens * (1. + 3. * (-vx - vy) + 4.5 * (vx + vy) * (vx + vy) - 1.5 * usq)
    feq[s, 8] = weight[8] * dens * (1. + 3. * (vx - vy) + 4.5 * (vx - vy) * (vx - vy) - 1.5 * usq)

def initialisation():
    global f1, f2, rho
    rho = np.zeros((nfluids, nx * ny))
    # Set viscosity
    tau[0] = tauA
    if nfluids == 2:
        tau[1] = tauB

    # Set interaction strength
    g[0, 0] = gA
    if nfluids == 2:
        g[1, 1] = gB
        g[0, 1] = g[1, 0] = gAB

    # Initialise 1-component system
    if nfluids == 1:
        for k in range(nx * ny):
            if ((k // nx - ny // 2) ** 2 + (k % nx - nx // 2) ** 2) <= radius ** 2:
                rho[0, k] = rhol  # liquid density
            else:
                rho[0, k] = rhog  # gas density

    # Initialise 2-component system
    if nfluids == 2:
        for y in range(ny):
            for x in range(nx):
                k = y * nx + x
                if y < ny // 2:
                    rho[0, k] = 0.1
                    rho[1, k] = 0.9
                else:
                    rho[0, k] = 0.9
                    rho[1, k] = 0.1

    # Initialise populations
    for s in range(nfluids):
        for k in range(nx * ny):
            ux[k] = uy[k] = 0.
            equilibrium(s, k)
            f1[s, k] = feq[s]
            f2[s, k] = feq[s]

def push():
    global f1, f2
    for s in range(nfluids):
        omega = 1. / tau[s]
        for y in range(ny):
            for x in range(nx):
                k = y * nx + x
                # Compute Guo's forcing terms
                for i in range(npop):
                    forcing[s, i] = weight[i] * (1. - 0.5 * omega) * ((3. * (cx[i] - ux[k]) + 9. * cx[i] * (cx[i] * ux[k] + cy[i] * uy[k])) * Fx[s, k] + (3. * (cy[i] - uy[k]) + 9. * cy[i] * (cx[i] * ux[k] + cy[i] * uy[k])) * Fy[s, k])
                # Compute equilibrium distributions
                equilibrium(s, k)
                # Collide and propagate
                for i in range(npop):
                    x2 = (x + cx[i] + nx) % nx
                    y2 = (y + cy[i] + ny) % ny
                    f2[s, y2 * nx + x2, i] = f1[s, k, i] * (1. - omega) + feq[s, i] * omega + forcing[s, i]
    f1, f2 = f2, f1

def write_file(name, data):
    plt.figure(figsize=(10, 10))
    plt.imshow(data.reshape(ny, nx), cmap='viridis')
    plt.colorbar()
    plt.title(name)
    plt.savefig(f'{name}.png')
    plt.close()
    files.download(f'{name}.png')

def write_profiles(step):
    compute_pressure()
    write_file(f'press_{step:05d}', press)
    write_file(f'velx_{step:05d}', ux)
    write_file(f'vely_{step:05d}', uy)
    for s in range(nfluids):
        write_file(f'density_comp{s}_{step:05d}', rho[s])

def calculate_delta_pressure():
    rho_gas = 0.
    rho_liq = 0.
    press_gas = 0.
    press_liq = 0.

    # Average gas pressure in a square box with 4 lattice nodes
    for x in range(4):
        for y in range(4):
            k = y * nx + x
            rho_gas += rho[0, k]
            press_gas += press[k]

    # Average liquid pressure in a square box with 4 lattice nodes
    for x in range(nx // 2 - 2, nx // 2 + 2):
        for y in range(ny // 2 - 2, ny // 2 + 2):
            k = y * nx + x
            rho_liq += rho[0, k]
            press_liq += press[k]

    rho_gas /= 16.
    rho_liq /= 16.
    press_gas /= 16.
    press_liq /= 16.

    delta_press = press_liq - press_gas
    rho_av = 0.5 * (rho_gas + rho_liq)
    y = ny // 2
    rad = 0

    for x in range(nx // 2):
        k = y * nx + x
        if rho[0, k] > rho_av:
            drho = rho[0, k] - rho[0, k-1]
            dx = (rho_av - rho[0, k-1]) / drho
            rad = (nx / 2. - x) + (1. - dx)
            break

    gamma = delta_press * rad

    print(f"  Densities: rho_g = {rho_gas:.6f}, rho_l = {rho_liq:.6f}")
    print(f"  Pressure difference: dp = {delta_press:.6f}")
    print(f"  Droplet radius: r = {rad:.6f}")
    print(f"  Surface tension: gamma = {gamma:.6f}")

def main():
    initialisation()

    for step in range(nsteps + 1):
        compute_density(f1)
        compute_sc_forces()
        compute_velocity(f1)
        push()

        if step % noutput == 0:
            print(f"Running time step {step}")
            write_profiles(step)
            calculate_delta_pressure()

if __name__ == "__main__":
    main()

Running time step 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  Densities: rho_g = 0.186790, rho_l = 1.688601
  Pressure difference: dp = 0.002751
  Droplet radius: r = 16.261085
  Surface tension: gamma = 0.044728


KeyboardInterrupt: 